
# <font color='056938'> **Librerias** </font>

In [28]:
from google.colab import drive
import sys
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [29]:
drive.mount('/content/drive')

#Define la parte del directorio que quieres trabajar
path = "/content/drive/MyDrive/Mod2/ANALITICA3"

sys.path.append(path)##para importar archivo de funciones propias a traves de import
os.chdir(path)## para que por defecto suba y descargue archivos partiendo de esa ruta
sys.path.append(f"{path}") #agragarle al path, poder leer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# <font color='056938'> **Cargar base de datos** </font>

In [30]:
con=sql.connect("data//db_movies")# conectarse a base de datos existente

In [31]:
cur=con.cursor()#otro tipo de conexión para ejecutar consultas en la base de datos sin traer ni llevar información

In [32]:
cur.execute("""SELECT name FROM sqlite_master WHERE type='table'""")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('peliculas_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

<font color='056938'> **Tabla de movies** </font>

In [33]:
df_movies=pd.read_sql("""SELECT * FROM movies""",con)
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


<font color='056938'> **Tabla de ratings** </font>

In [34]:
df_ratings=pd.read_sql("""SELECT * FROM ratings""",con)
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


<font color='056938'> **1. Verificar datos faltantes** </font>

In [35]:
# Muestra las variables que tienen datos faltantes en la base de datos calculo diferencial
df_movies.isnull().any()

,0
movieId,False
title,False
genres,False


In [36]:
# Muestra las variables que tienen datos faltantes en la base de datos calculo diferencial
df_ratings.isnull().any()

,0
userId,False
movieId,False
rating,False
timestamp,False


No se evidencia datos faltantes en ninguna de las dos tablas

 <font color='056938'> **2. Verificar datos duplicados** </font>

In [37]:
# Identificar las filas duplicadas incluyendo todas las filas duplicadas
duplicados_movies = df_movies.duplicated(keep=False)

# Filtrar y mostrar las filas duplicadas
filas_duplicadas_movies = df_movies[duplicados_movies]

filas_duplicadas_movies

,movieId,title,genres


In [38]:
# Identificar las filas duplicadas incluyendo todas las filas duplicadas
duplicados_ratings = df_ratings.duplicated(keep=False)

# Filtrar y mostrar las filas duplicadas
filas_duplicadas_ratings = df_ratings[duplicados_ratings]

filas_duplicadas_ratings

,userId,movieId,rating,timestamp


Las dos tablas no tienen datos duplicados

<font color='056938'> **3. Verificar outliers** </font>

Observar distribución de la tabla ratings

**a) Calcular la distribución de calificaciones**

In [39]:
ratings_count = pd.read_sql("""
    SELECT
        "rating" AS Rating,
        COUNT(*) AS conteo
    FROM ratings
    GROUP BY "rating"
    ORDER BY conteo desc
""", con)

data  = go.Bar( x=ratings_count.Rating,y=ratings_count.conteo, text=ratings_count.conteo, textposition="outside")
Layout=go.Layout(title="Count of ratings",xaxis={'title':'Rating'},yaxis={'title':'Count'})
go.Figure(data,Layout)

Se observa que todas las películas han recibido calificaciones, sin registrarse valores de cero. Además,la mayoria de las peliculas cuentan con buena calificación

**b) Calcular la cantidad de calificaciones que  hizo cada usuario**

In [40]:
# conteo de calificaciones por cada usuario
user_ratings_count = pd.read_sql("""
    SELECT
        "userId" AS user_id,
        COUNT(*) AS conteo_calificaciones
    FROM ratings
    GROUP BY "userId"
    ORDER BY conteo_calificaciones asc
""", con)

# gráfica
calificaciones_user = go.Bar(x=user_ratings_count.user_id, y=user_ratings_count.conteo_calificaciones,
              text=user_ratings_count.conteo_calificaciones, textposition="outside")
layout = go.Layout(title="Conteo de Calificaciones por Usuario",
                   xaxis={'title': 'User ID'},
                   yaxis={'title': 'Conteo de Calificaciones'})
go.Figure(data=[calificaciones_user ], layout=layout)

In [41]:
# boxplot para visualizar la distribución de las calificaciones por usuario
box_user_rating = go.Box(
    y=user_ratings_count.conteo_calificaciones,  # Datos de las calificaciones
    name="Conteo de Calificaciones por Usuario"
)

layout = go.Layout(
    title="Boxplot: Distribución de las Calificaciones por Usuario",
    yaxis={'title': 'Conteo de Calificaciones'},
)

# Mostrar el gráfico
go.Figure(data=[box_user_rating], layout=layout)

En los gráficos se observa la presencia de outliers, donde pocos usuarios registran más de 500 calificaciones. Para reducir el sesgo que esto podría generar, se crea una tabla que incluye únicamente a los usuarios que han calificado más de 50 y menos de 500 películas.

Tabla de usuarios con más de 50 peliculas calificadas y menos de 500

In [42]:
with con:
    con.execute("""
        DROP TABLE IF EXISTS usuarios_sel;
    """)
    con.execute("""
        CREATE TABLE usuarios_sel AS
        SELECT
            "userId" AS user_id,
            COUNT(*) AS conteo_calificaciones
        FROM ratings
        GROUP BY "userId"
        HAVING conteo_calificaciones > 50 AND conteo_calificaciones <= 500
        ORDER BY conteo_calificaciones ASC
    """)

# 2. Leer los datos de la tabla usuarios_sel
user_ratings_count2 = pd.read_sql("SELECT * FROM usuarios_sel", con)

# boxplot para visualizar la distribución de las calificaciones por usuario
box_user_rating2 = go.Box(
    y=user_ratings_count2.conteo_calificaciones,  # Datos de las calificaciones
    name="Conteo de Calificaciones por Usuario"
)

layout = go.Layout(
    title="Boxplot: Distribución de las Calificaciones por Usuario",
    yaxis={'title': 'Conteo de Calificaciones'},
)

# Mostrar el gráfico
go.Figure(data=[box_user_rating2], layout=layout)

**c) Calcular cuantas calificaciones tiene cada pelicula**

In [43]:
# Consulta para obtener el número de usuarios que han visto cada película
movies_rating = pd.read_sql("""
    SELECT
        "movieId" AS movie_id,
        COUNT(*) AS conteo_peliculas
    FROM ratings
    GROUP BY "movieId"
    ORDER BY conteo_peliculas desc
""", con)

# boxplot para visualizar la distribución de las calificaciones por usuario
box_movie_rat = go.Box(
    y=movies_rating.conteo_peliculas,  # Datos de las calificaciones
    name="Conteo de peliculas vistas"
)

layout = go.Layout(
    title="Boxplot: Distribución de las peliculas calificadas",
    yaxis={'title': 'Conteo de peliculas calificadas'},
)

# Mostrar el gráfico
go.Figure(data=[box_movie_rat], layout=layout)

En el boxplot se evidencia la presencia de varios outliers, donde algunas películas tienen un mínimo de una calificación. Dado que para generar recomendaciones precisas se requiere que una pelicula tenga varias recomendaciones de diferentes usuarios, por tanto, se excluyen aquellas películas que no cuenten con más de 50 calificaciones.

Tabla de peliculas con más de 50 peliculas calificaciones

In [44]:
with con:
    con.execute("""
        DROP TABLE IF EXISTS peliculas_sel;
    """)
    con.execute("""
        CREATE TABLE peliculas_sel AS
        SELECT
            "movieId" AS movie_id,
            COUNT(*) AS conteo_peliculas
        FROM ratings
        GROUP BY "movieId"
        HAVING conteo_peliculas >= 50
        ORDER BY conteo_peliculas DESC
    """)

# 2. Leer los datos de la tabla peliculas_sel
movies_rating2 = pd.read_sql("SELECT * FROM peliculas_sel", con)

# 3. Gráfico boxplot para la distribución
box_movie_rat2 = go.Box(
    y=movies_rating2.conteo_peliculas,  # Datos de conteo de películas
    name="Conteo de películas calificadas"
)

layout = go.Layout(
    title="Boxplot: Distribución de las películas calificadas",
    yaxis={'title': 'Conteo de películas calificadas'},
)

# Mostrar el gráfico
fig = go.Figure(data=[box_movie_rat2], layout=layout)
fig.show()

**d)Crear tabla completa para utilizar en los modelos**



Crea una nueva tabla llamada ratings_final que contiene: el ID del usuario, el ID de la película, la calificación dada y la fecha/hora de la calificación. Pero solo para los usuarios que están en usuarios_sel y las películas que están en peliculas_sel.


In [45]:
# Tabla 'ratings_final' que combina 'ratings' con 'usuarios_sel' y 'peliculas_sel'
with con:
    con.execute("DROP TABLE IF EXISTS ratings_final")

with con:
    con.execute("""
        CREATE TABLE ratings_final AS
        SELECT a."userId" AS user_id,
               a."movieId" AS movie_id,
               a."rating" AS movie_rating,
               a."timestamp" AS movie_timestamp
        FROM ratings a
        INNER JOIN usuarios_sel b ON a."userId" = b.user_id
        INNER JOIN peliculas_sel c ON a."movieId" = c.movie_id
    """)

# Visualizar la tabla 'ratings_final'
ratings_final = pd.read_sql("SELECT * FROM ratings_final", con)
ratings_final.head()

,user_id,movie_id,movie_rating,movie_timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Crea una nueva tabla llamada movies_final que contiene: El ID de la película, el nombre de la película, los géneros de la película, pero solo con las películas que aparecen en peliculas_sel.

In [46]:
# Crear tabla 'movies_final' basada en 'movies' y 'peliculas_sel'
with con:
    con.execute("DROP TABLE IF EXISTS movies_final")

with con:
    con.execute("""
        CREATE TABLE movies_final AS
        SELECT a."movieId" AS movie_id,
               a."title" AS movie_title,
               a."genres" AS movie_genres
        FROM movies a
        INNER JOIN peliculas_sel b ON a."movieId" = b.movie_id
    """)

# Visualizar la tabla 'movies_final'
movies_final = pd.read_sql("SELECT * FROM movies_final", con)
movies_final.head()

,movie_id,movie_title,movie_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,6,Heat (1995),Action|Crime|Thriller
4,7,Sabrina (1995),Comedy|Romance


Con las tablas previamente limpiadas, se procede a crear una tabla consolidada que incluya el ID del usuario, el ID de la película, el nombre de la película, su género, la calificación y la fecha/hora de la calificación. Esta tabla se genera la información de las calificaciones en la tabla ratings_final y la información de las películas de la tabla movies_final, usando un INNER JOIN para unir las tablas donde coincidan con el ID de la pelicula.

In [47]:
with con:
    con.execute("""
        DROP TABLE IF EXISTS full_ratings;
    """)
    con.execute("""
        CREATE TABLE full_ratings AS
        SELECT
            a.user_id,
            a.movie_id,
            b.movie_title,
            b.movie_genres,
            a.movie_rating,
            a.movie_timestamp
        FROM ratings_final a
        INNER JOIN movies_final b ON a.movie_id = b.movie_id
    """)

# 2. Leer la tabla full_ratings
tabla_completa = pd.read_sql("SELECT * FROM full_ratings", con)

# 3. Visualizar
tabla_completa

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703
1,1,3,Grumpier Old Men (1995),Comedy|Romance,4.0,964981247
2,1,6,Heat (1995),Action|Crime|Thriller,4.0,964982224
3,1,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,5.0,964983815
4,1,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,5.0,964982931
...,...,...,...,...,...,...
25973,607,3785,Scary Movie (2000),Comedy|Horror,2.0,964743188
25974,607,3793,X-Men (2000),Action|Adventure|Sci-Fi,3.0,964743155
25975,607,3994,Unbreakable (2000),Drama|Sci-Fi,3.0,997847202
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",Action|Drama|Romance,2.0,997847173


In [48]:
## al cruzar tablas a veces se duplican registros, por tanto se identifican las filas duplicadas incluyendo todas las filas duplicadas
duplicados_completa = tabla_completa.duplicated(keep=False)

# Filtrar y mostrar las filas duplicadas
filas_duplicadas_completa = tabla_completa[duplicados_completa]

filas_duplicadas_completa

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp


No hay datos duplicados

Para facilitar la construcción de los modelos de recomendación, se extrae el año de estreno desde el campo movie_title y se crea una nueva columna denominada movie_year.

In [49]:
# Extraer el año desde movie_title y crear la nueva columna movie_year
tabla_completa['movie_year'] = tabla_completa['movie_title'].str.extract(r'\((\d{4})\)', expand=False)

# Convertir a numérico
tabla_completa['movie_year'] = pd.to_numeric(tabla_completa['movie_year'], errors='coerce')

In [50]:
tabla_completa['movie_title'] = tabla_completa['movie_title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
tabla_completa.head()

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703,1995
1,1,3,Grumpier Old Men,Comedy|Romance,4.0,964981247,1995
2,1,6,Heat,Action|Crime|Thriller,4.0,964982224,1995
3,1,47,Seven (a.k.a. Se7en),Mystery|Thriller,5.0,964983815,1995
4,1,50,"Usual Suspects, The",Crime|Mystery|Thriller,5.0,964982931,1995


In [51]:
# Guardar la columna incluida en la tabla full_rating
tabla_completa.to_sql('full_ratings', con, if_exists='replace', index=False)
tabla_completa

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703,1995
1,1,3,Grumpier Old Men,Comedy|Romance,4.0,964981247,1995
2,1,6,Heat,Action|Crime|Thriller,4.0,964982224,1995
3,1,47,Seven (a.k.a. Se7en),Mystery|Thriller,5.0,964983815,1995
4,1,50,"Usual Suspects, The",Crime|Mystery|Thriller,5.0,964982931,1995
...,...,...,...,...,...,...,...
25973,607,3785,Scary Movie,Comedy|Horror,2.0,964743188,2000
25974,607,3793,X-Men,Action|Adventure|Sci-Fi,3.0,964743155,2000
25975,607,3994,Unbreakable,Drama|Sci-Fi,3.0,997847202,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action|Drama|Romance,2.0,997847173,2000


Esta tabla de full ratings se utilizará para realizar los modelos

In [52]:
def Tabla_procesada(conn):
    # Cargar la tabla
    tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

    # Escalar el año
    sc = StandardScaler()
    tabla_completa[["year_sc"]] = sc.fit_transform(tabla_completa[['movie_year']])

    # Separar géneros y expandir
    df_genres = tabla_completa.assign(movie_genres=tabla_completa['movie_genres'].str.split('|')).explode('movie_genres')
    df_genres = df_genres.rename(columns={'movie_genres': 'genre'})

    # Eliminar columnas innecesarias
    movie_dum1 = df_genres.drop(columns=['user_id', 'movie_rating', 'movie_timestamp', 'movie_year', 'movie_title'])

    # Dumificación
    movie_dum2 = pd.get_dummies(movie_dum1, columns=['genre'])

    # Unificar por movie_id
    movie_unificado = movie_dum2.groupby('movie_id', as_index=False).max()

    # También devolver tabla original para nombres de películas
    return movie_unificado, tabla_completa